# Снижение размерности

## Пример 1.

Давайте для начала рассмотрим следующий пример:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = np.array([[2,3,4,7,10], [1,1,1,1,1]]).T

In [ ]:
X

У нас есть 5 наблюдений и два признака, вопрос, какой из признаков
вам кажется **важнее** и почему?

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);

Естественным ответом кажется сказать что первый, потому что этот признак **изменяется** от объекта к объекту,
а второй признак - нет. 

Вопрос: Что значит **изменяется**?

Хорошо, давайте теперь на другой пример:

In [ ]:
X = np.array([[2,3,4,7,10], [4,6,8,14,20]]).T
# X = np.array([[2,3,4,7,10], [2,3,4,7,10]]).T

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);

Какой из признаков важнее здесь?

Ответить уже не так легко, изменение происходит как по первой координате, так и по второй, но что если мы повернем картинку (скажем на 60 градусов)?

In [ ]:
alpha = np.pi / 3
rot_matrix = np.array([[np.cos(alpha), -np.sin(alpha)],
                      [np.sin(alpha), np.cos(alpha)]])

X_rot = X.dot(rot_matrix)

In [ ]:
X_rot

In [ ]:
plt.scatter(X[:, 0], X[:, 1], label='X original');
plt.scatter(X_rot[:, 0], X_rot[:, 1], label='X rotated on 60 degrees');
plt.legend();

Можем ли мы теперь сказать какой из признаков для нас **важнее**?

Итак за словами **изменяется** и **важнее** на самом деле скрывается слово **Дисперсия**.

Посмотрим на еще один пример:

In [ ]:
random = np.random.RandomState(7)
time_steps = np.linspace(0, 10, 40).reshape(-1, 1)
X = time_steps + 2*random.normal(scale = .6, size=(40,1))
X = np.concatenate((time_steps, X), axis=1 )

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);
plt.xlabel('Признак 1')
plt.ylabel('Признак 2');

In [ ]:
np.std(X, axis=0)

Как видно дисперсия внутри **Признака 1** немногим меньше дисперсии внутри **Признака 2**.

### Вопрос

Существует ли какая то **их линейная комбинация**: какой то новый признак, дисперсия внутри которого будет **больше**?

## Ответ

да существует!

In [ ]:
plt.scatter(X[:, 0], X[:, 1]);
plt.xlabel('Признак 1')
plt.ylabel('Признак 2');
plt.plot([0, 10], [0, 10], c='r');

А что это за направление?

В нашем случае это простой поворот на 45 градусов

In [ ]:
alpha = np.pi / 4
rot_matrix = np.array([[np.cos(alpha), -np.sin(alpha)],
                      [np.sin(alpha), np.cos(alpha)]])

X_rot = X.dot(rot_matrix)

In [ ]:
plt.figure(figsize=(12, 3))
plt.scatter(X_rot[:, 0], X_rot[:, 1]);
plt.xlabel('Признак 1')
plt.ylabel('Признак 2');

In [ ]:
print('Дисперсия 1 признака: {:.3f}; Дисперсия 2 признака: {:.3f}'.format(X[:, 0].std(), 
                                                                          X[:, 1].std()))

print('Дисперсия 1 компоненты: {:.3f}; Дисперсия 2 компоненты: {:.3f}'.format(X_rot[:, 0].std(), 
                                                                              X_rot[:, 1].std()))

# PCA

метод который называется Principal Component Analysis позволяет выделять эти компоненты.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(2)
xpca = pca.fit_transform(X)

In [ ]:
pca.components_

Сравните эти компоненты с матрицей поворота, есть ли отличия?

In [ ]:
print('Дисперсия 1 компоненты: {:.3f}; Дисперсия 2 компоненты: {:.3f}'.format(X.dot(pca.components_[0]).std(),
                                                                              X.dot(pca.components_[1]).std()))

Как видно **PCA** нашел лучшие направления чем поворот на 45 градусов.

### Оказывается для наших данных достаточно измерить только 1 признак, 2ой же признак является избыточным!

----

До сих пор мы смотрели на двумерные примеры и из двумерных данных путем преобразования системы координат получали новые двумерные данные. Давайте рассмотрим пример посложнее:

## Пример 2.
Хорошо давайте теперь рассмотрим другой пример

![pca](pca_example.png)


>Pretend we are studying the motion
of the physicist’s ideal spring. This system consists of a ball
of mass `m` attached to a massless, frictionless spring. The ball
is released a small distance away from equilibrium (i.e. the
spring is stretched). Because the spring is ideal, it oscillates
indefinitely along the x-axis about its equilibrium at a set frequency.
However, being ignorant experimenters we do not know any of this. We do not know which, let alone how many, axes
and dimensions are important to measure. Thus, we decide to measure the ball’s position in a three-dimensional space (since we live in a three dimensional world). Specifically, we place three movie cameras around our system of interest. At 120 Hz each movie camera records an image indicating a two dimensional position of the ball (a projection). Unfortunately, because of our ignorance, we do not even know what are the real `x`, `y` and `z` axes, so we choose **three camera positions a,b and c at some arbitrary angles** with respect to the system. The angles
between our measurements might not even be 90 degrees! Now, we record with the cameras for several minutes. The big question remains: how do we get from this data set to a simple equation of `x`?

**A Tutorial on Principal Component Analysis** https://arxiv.org/pdf/1404.1100.pdf 

In [ ]:

def generate_points(random_state=331):
    random = np.random.RandomState(random_state)
    time_steps = np.linspace(0, 10, 40).reshape(-1, 1)
    X = time_steps + 3*random.normal(scale=0.3, size=(40,1))
    X = np.concatenate((time_steps, X), axis=1 )

    alpha = -np.pi/2
    beta = np.pi/4
    rot_matrix_1 = np.array([[np.cos(alpha), -np.sin(alpha)],
                          [np.sin(alpha), np.cos(alpha)]])
    rot_matrix_2 = np.array([[np.cos(beta), -np.sin(beta)],
                          [np.sin(beta), np.cos(beta)]])

    X_rot1 = X.dot(rot_matrix_1)
    X_rot2 = X.dot(rot_matrix_2) + 5
    
    return X, X_rot1, X_rot2

In [ ]:
# from utils import generate_points

In [ ]:
camera1, camera2, camera3 = generate_points()

In [ ]:
plt.figure(figsize=(7,7))
ax1 = plt.subplot(2,2,1)
ax1.scatter(camera1[:, 0], camera1[:, 1], label='Camera 1', c='b')
ax1.legend();
ax2 = plt.subplot(2,2,2)
ax2.scatter(camera2[:, 0], camera2[:, 1], label='Camera 2', c='r')
ax2.legend();
ax3 = plt.subplot(2,2,3)
ax3.scatter(camera3[:, 0], camera3[:, 1], label='Camera 3', c='orange')
ax3.legend();

In [ ]:
X = np.concatenate((camera1, camera2, camera3), axis=1)

In [ ]:
X.shape

In [ ]:
# X =(X - X.mean(axis=0))/X.std(axis=0)

Итак для каждого наблюдения мы располагаем **6 измерениями** (2 координаты с каждой камеры). 
Но мы то знаем что на самом деле пружина изменялась только вдоль **одного единственного направления**!

In [ ]:
pca = PCA(2)
X_pca = pca.fit_transform(X)

In [ ]:
plt.figure(figsize=(15, 2))
plt.scatter(X_pca[:, 0], X_pca[:, 1]);
plt.xlabel('Principle Component 1')
plt.ylabel('Principle Component 2');

In [ ]:
for idx, feature in enumerate(X.T):
    print('Дисперсия признака {}: {:.3f}'.format(idx+1, feature.std()))

In [ ]:
for idx, feature in enumerate(X_pca.T):
    print('Дисперсия компоненты {}: {:.3f}'.format(idx+1, feature.std()))

Таким образом метод главных компонент позволяет **снизить размерность данных**, преобразуя старые признаки в новые.

## PCA важное

![pca](pca2.png)



$$ X_{n \times m} \rightarrow X^{\text{reduced}}_{n \times k},\ \ \  \text{s.t.} \ \ \  k < m$$
                        
$$X_{n \times m} \cdot S_{m\times k} = X^{\text{reduced}}_{n \times k}$$



1. PCA проецирует данные на новый признаки, которые являются линейной комбинацией старых.
2. Новые признаки называются компонентами (или Главными компонентами). Эти направления попарно ортогональны.
3. Мерой важности новых признаков является **доля объясненнной дисперсии**.
4. Перед применением метода Главных компонент данные необходимо центрировать

---

* Подробный туториал по методу главных компонент https://arxiv.org/pdf/1404.1100.pdf (на английском)
* Серия из 3 видео посвященных более подробному разбору в том числе пример вычисления руками (на русском) 
    * https://www.youtube.com/watch?v=NKmwnILrHD8&t=
    * https://www.youtube.com/watch?v=cgdnlSv6kpg
    * https://www.youtube.com/watch?v=WP2VLhAAM24&t=
* Статья из цикла статей открытого курса по машинному обучению https://habr.com/ru/company/ods/blog/325654/ (на русском)
    
    
----

В пакете sklearn есть специальный класс PCA который реализует метод главных компонент.

основные параметры, которые можно увидеть после обучения:
* `.components_` - новые компоненты (количество - n_components). Каждая компонента - линейная комбинация изначальных признаков (представлна вектором длинной изначального количества признаков).

Например если X имеет размер $100 \times 5$, и мы хотим найти 3 главные компоненты, то после обучения поле `.components_` будет содержать матрицу размера $3 \times 5$, т.е. задача PCA найти такое P, что:


* `.explained_variance_` и `.explained_variance_ratio_` - объясненная дисперсия и доля объясненной дисперсии каждой компонентой.

## Пример 3. Digits.

Давайте посмотрим на то насколько наши рукописные цифры с 64 признаками можно хорошо спроецировать на 2 измерения.

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
X, y = load_digits(n_class=2, return_X_y=True)

# X = np.vstack([X[y==4], X[y==5]])
# y = [4]*181+[5]*182

In [ ]:
pca = PCA(2)

X_pca = pca.fit_transform(X)

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1]);

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y);

In [ ]:
new_0 = [-20, 0] # 0,1
# new_0 = [-20, 20] # 4,5
plt.imshow(pca.inverse_transform(new_0).reshape(8,8));

In [ ]:
new_0 = [20, -10] # 0,1
# new_0 = [20, -10] # 4,5
plt.imshow(pca.inverse_transform(new_0).reshape(8,8));

In [ ]:
plt.imshow(np.random.rand(64).reshape(8,8));

In [ ]:
# y = 0.25*x + 5 # 0,1
# y = -0.75*x + 5 # 4,5

x_coords = np.linspace(-20, 20, 20)
y_coords = 0.25 * x_coords + 5
# y_coords = -0.75 * x_coords + 5
xy_coords = np.vstack([x_coords, y_coords]).T
new_digits = pca.inverse_transform(xy_coords).reshape(20, 8, 8)

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(8,8))

for i, ax in enumerate(axs.flatten()):
    ax.imshow(np.clip(new_digits[i], 4, 10), cmap='gray')

* Попробуйте разбить данные на две части: train и test. Обучить преобразование координат с помощью PCA на одной части и потом спроецировать вторую часть данных на найденные компоненты. Сравните с результатом полученным без разделения данных на две части. Постройте доверительный интервал для первой главной компоненты используя метод бутстреппинга. 

* Попробуйте увеличить количество классов (параметр n_class в функции load_digits) и посмотреть как изменятся наши компоненты.

# Другие методы снижения размерности

TSNE, MDS, LDA, Random Projection, Autoencoders, ...

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state=33)
X_tsne = tsne.fit_transform(X)

plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y);

# tsne = TSNE(random_state=32)
# X_tsne = tsne.fit_transform(X)

# plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y);

# Для чего используют методы снижения размерности?

$$ X_{n \times m} \rightarrow X^{\text{reduced}}_{n \times k},\ \ \  \text{s.t.} \ \ \  k < m$$
                        

- Снижение уровня шума
- Для визуализации
- Для сэмплирования данных
- Для детектирования аномалий
- ...